In [1]:
import scanpy as sc
import dynamo as dyn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
dyn.__version__

'1.4.0'

In [2]:
color_map = {
    '1':'#a6cee3',
    '2':'#54278f',
    '3':'#e7298a',
    '4':'#1f77b4',
    '5':'#ff6600'
}

In [3]:
adata = sc.read_h5ad('/syn1/liangzhen/jinhua_jilab_project/result/scRNA/h5ad/adata.h5ad')
cell_lineage = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/cell_lineage.T1_T2_T3a.csv',header=0,index_col=0)
adata = adata[adata.obs.cellname.isin(cell_lineage.index),]
adata.obs['lineageGrp'] = cell_lineage.loc[adata.obs['cellname'],'lineageGrp'].to_list()
adata.obs['lineageGrp'] = ['C'+str(lineageGrp) for lineageGrp in adata.obs['lineageGrp']]
adata.obs['sample'] = ['_'.join(sample.split('_')[0:2]) for sample in adata.obs['Sample']]

/tmp/ipykernel_386961/2270980215.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['lineageGrp'] = cell_lineage.loc[adata.obs['cellname'],'lineageGrp'].to_list()


In [4]:
metadata = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/scRNA/cellranger/metadata.T1_T2_T3a.harmony.final.refine.csv',header=0,index_col=0)
metadata.index = metadata['cellName'].to_list()

/tmp/ipykernel_386961/1034179230.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/scRNA/cellranger/metadata.T1_T2_T3a.harmony.final.refine.csv',header=0,index_col=0)


In [5]:
adata = adata[adata.obs.cellname.isin(metadata.cellName),]

In [6]:
adata.obsm['X_umap'] = np.array(metadata.loc[adata.obs.cellname,['umapharmony_1','umapharmony_2']])
adata.obs['cluster'] = metadata.loc[adata.obs.cellname,'seurat_clusters_rename'].to_list()
adata.obs['cluster'] = adata.obs['cluster'].astype('str')
adata.obs['experiment'] = [sample.split('_')[1] for sample in adata.obs['Sample']]
adata.obs['time'] = [ sample.split('_')[0] for sample in adata.obs['sample']]

/tmp/ipykernel_386961/343543299.py:1: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata.obsm['X_umap'] = np.array(metadata.loc[adata.obs.cellname,['umapharmony_1','umapharmony_2']])


In [7]:
transition_files = os.listdir('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/cell_transition/')
transition_files = [ filename for filename in transition_files if filename.startswith('C')]

In [8]:
transition_files

['C35-153_cell_transition_ALL_S2_S3a.csv',
 'C49-194_cell_transition_ALL_S2_S3a.csv',
 'C6-1249_cell_transition_ALL_S2_S3a.csv',
 'C40-270_cell_transition_ALL_S2_S3a.csv',
 'C28-540_cell_transition_ALL_S2_S3a.csv',
 'C46-183_cell_transition_ALL_S2_S3a.csv',
 'C32-298_cell_transition_ALL_S2_S3a.csv',
 'C26-296_cell_transition_ALL_S2_S3a.csv',
 'C14-573_cell_transition_ALL_S2_S3a.csv',
 'C8-891_cell_transition_ALL_S2_S3a.csv',
 'C2-4397_cell_transition_ALL_S2_S3a.csv',
 'C30-233_cell_transition_ALL_S2_S3a.csv',
 'C10-793_cell_transition_ALL_S2_S3a.csv',
 'C36-374_cell_transition_ALL_S2_S3a.csv',
 'C16-577_cell_transition_ALL_S2_S3a.csv',
 'C42-178_cell_transition_ALL_S2_S3a.csv',
 'C69-123_cell_transition_ALL_S2_S3a.csv',
 'C7-2304_cell_transition_ALL_S2_S3a.csv',
 'C11-400_cell_transition_ALL_S2_S3a.csv',
 'C82-171_cell_transition_ALL_S2_S3a.csv',
 'C15-405_cell_transition_ALL_S2_S3a.csv',
 'C20-341_cell_transition_ALL_S2_S3a.csv',
 'C27-255_cell_transition_ALL_S2_S3a.csv',
 'C95-105_ce

In [9]:
for transition_file in transition_files:
    print(transition_file)
    transition = pd.read_csv('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/cell_transition/'+transition_file,index_col=0)
    transition.index = transition.index.astype('str')
    transition.columns = transition.index
    lineageGrp = transition_file.split('-')[0]
    clusters = transition.index.astype('str')
    adata_selected = adata[(adata.obs['lineageGrp']==lineageGrp) & 
                           (adata.obs['cluster'].isin(clusters)) &
                           (adata.obs['time'].isin(['T2','T3'])) 
                          ]
    colors = [color_map[cluster] for cluster in transition.index]
    transition = transition.loc[adata_selected.obs['cluster'].unique(),adata_selected.obs['cluster'].unique()]
    adata_selected.obsm['X_umap'] = np.array(adata_selected.obsm['X_umap'])
    
    dyn.pl.state_graph(adata_selected,pointsize=1.5,facecolor='#000000',edgecolor='#000000',#edge_scale=15,
                   color=['cluster'],
                   group='cluster',
                   basis='umap',
                   state_graph=transition.to_numpy(),
                   #method='vf',
                   figsize=[4,4],
                   color_key=colors,#frontier =True
                   save_show_or_return='save',
                   save_kwargs = {'path': '/syn1/liangzhen/jinhua_jilab_project/result/Figures/Figure3/transition_map/', 'prefix': lineageGrp,'ext': 'svg',
                                 "transparent": True, "close": True, "verbose": True}
                  )
    #plt.title(transition_file)
    #break

C35-153_cell_transition_ALL_S2_S3a.csv
|-----------> plotting with basis key=X_umap
|-----------> skip filtering cluster by stack threshold when stacking color because it is not a numeric type


/tmp/ipykernel_386961/95944344.py:14: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_selected.obsm['X_umap'] = np.array(adata_selected.obsm['X_umap'])


Saving figure to /syn1/liangzhen/jinhua_jilab_project/result/Figures/Figure3/transition_map/C35_dyn_savefig.svg...
Done
C49-194_cell_transition_ALL_S2_S3a.csv
|-----------> plotting with basis key=X_umap
|-----------> skip filtering cluster by stack threshold when stacking color because it is not a numeric type
Saving figure to /syn1/liangzhen/jinhua_jilab_project/result/Figures/Figure3/transition_map/C49_dyn_savefig.svg...


/tmp/ipykernel_386961/95944344.py:14: ImplicitModificationWarning: Setting element `.obsm['X_umap']` of view, initializing view as actual.
  adata_selected.obsm['X_umap'] = np.array(adata_selected.obsm['X_umap'])


Done
C6-1249_cell_transition_ALL_S2_S3a.csv
|-----------> plotting with basis key=X_umap
|-----------> skip filtering cluster by stack threshold when stacking color because it is not a numeric type


...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>